In [1]:
# Set up data collection foundation

TITLE=[]
GENRE=[]
CREATED_BY=[]
STARRING=[]
NETWORK=[]
RUNNING_TIME=[]
NUM_SEASONS=[]
NUM_EPISODES=[]

TABLE_COLUMNS = ['title', 'genre', 'created_by', 'starring', 'network', 'running_time', 'num_seasons', 'num_episodes']

DATABASE = {'title':TITLE, 'genre':GENRE, 'created_by':CREATED_BY, 'starring':STARRING, 'network':NETWORK, 'running_time':RUNNING_TIME, 'num_seasons':NUM_SEASONS, 'num_episodes':NUM_EPISODES}

In [2]:
from bs4 import BeautifulSoup as soup
import pandas as pd
import re
from urllib import request

# Convert html to a more user friendly format
def soupTheLink(url):
    html = request.urlopen(url).read().decode('utf8')
    return soup(html, 'lxml')

# Collect a list of Wikipedia links to all American TV shows
def collectLinks(url):
    souped = soupTheLink(url)
    
    showLinks = []
    
    for listItem in souped.find_all('li'):
        link = listItem.find('a', href = True)
        try:
            fullLink = 'https://en.wikipedia.org' + link['href']
            showLinks.append(fullLink)
        except TypeError:
            pass
    showLinks = showLinks[28:2282]
    return showLinks

# Collect data from Wikipedia links
def compileData(listOfShowLinks):
    print("number of shows = ", len(listOfShowLinks))
    count = 0
    for url in listOfShowLinks[923:1054]:
        count += 1
        print('show number ', count)
        souped = soupTheLink(url)

        table_num = 0
        for table in souped.find_all(class_='infobox'):
            table_num += 1
            if table_num == 1:
                has_genre = 0
                has_created_by = 0
                has_starring = 0
                has_network = 0
                has_release = 0
                has_running_time = 0
                has_num_seasons = 0
                has_num_episodes = 0

                for row in table.find_all('tr'):

                    for category in row.find_all('th'):
                        feature = category.contents

                        if feature == ['Genre']:
                            has_genre = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                for genre in info.find_all('a'):
                                    rowContent.extend(genre.contents)
                            print('Genre: ', rowContent)

                            if rowContent == []:
                                for info in row.find_all('td'):
                                    rowContent.extend(info.contents)
                                print('Genre: ', rowContent)

                            DATABASE['genre'].append(rowContent)

                        elif feature == ['Created by'] or feature == ['Written by'] and has_created_by == 0:
                            has_created_by = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                creators = []

                                for creator in info.find_all('a'):
                                    name = str(creator.contents)
                                    if name[2] != '[':
                                        creators.extend(creator.contents)
                                rowContent.extend(creators)
                            print('Created by: ', rowContent)

                            if rowContent == []:
                                for info in row.find_all('td'):
                                    rowContent.extend(info.contents)
                                rowContent = rowContent[0]
                                print('Created by: ', rowContent)

                            DATABASE['created_by'].append(rowContent)

                        elif feature == ['Starring']:
                            has_starring = 1

                            rowContent = []

                            for info in row.find_all('td'):

                                for stars in info.find_all('a'):
                                    rowContent.extend(stars.contents)

                            print('Starring: ', rowContent)

                            if rowContent == []:
                                for info in row.find_all('td'):
                                    rowContent.extend(info.contents)
                                rowContent = rowContent[0]
                                print('Starring: ', rowContent)

                            DATABASE['starring'].append(rowContent)

                        elif feature == ['Original network']:
                            has_network = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                for network in info.find_all('a'):
                                    rowContent.extend(network.contents)
                            print('Network: ', rowContent)

                            DATABASE['network'].append(rowContent)

                        elif feature == ['Running time']:
                            has_running_time = 1

                            rowContent = []

                            for info in row.find_all('td'):
                                rowContent = info.contents
                                print('Running time: ', rowContent)

                                rowContent = rowContent[0]
                                rowContent = rowContent[0:2]

                                DATABASE['running_time'].append(rowContent)

                        elif feature[-1] == ' of seasons':
                            has_num_seasons = 1

                            for info in row.find_all('td'):
                                rowContent = info.contents[0]
                                print('Seasons: ', rowContent)

                                DATABASE['num_seasons'].append(rowContent)

                        elif feature[-1] == ' of episodes':
                            has_num_episodes = 1

                            for info in row.find_all('td'):
                                rowContent = info.contents[0]
                                print('Episodes:', rowContent)
                                DATABASE['num_episodes'].append(rowContent)


                        else:
                            continue
                    
                if has_genre == 0:
                    print('Genre: None')
                    DATABASE['genre'].append("None")
                if has_created_by == 0:
                    print('Created by: None')
                    DATABASE['created_by'].append("None")
                if has_starring == 0:
                    print('Starring: None')
                    DATABASE['starring'].append("None")
                if has_network == 0:
                    print('Network: None')
                    DATABASE['network'].append("None")
                if has_running_time == 0:
                    print('Running time: None')
                    DATABASE['running_time'].append("None")
                if has_num_seasons == 0:
                    print('Seasons: None')
                    DATABASE['num_seasons'].append("None")
                if has_num_episodes == 0:
                    print('Episodes: None')
                    DATABASE['num_episodes'].append("None")
        if table_num > 0:
            titleCount = 0
            for title in souped.find_all(class_='firstHeading'):
                for content in title.find_all('i'):
                    titleCount = 1
                    showTitle = content.contents
                    print(showTitle)
                    DATABASE['title'].append(showTitle)
                if titleCount == 0:
                    showTitle = title.contents
                    print(showTitle)
                    DATABASE['title'].append(showTitle)

In [4]:
# Main function
mainUrl = 'https://en.wikipedia.org/wiki/List_of_American_television_series'
links = collectLinks(mainUrl)
compileData(links)
df = pd.DataFrame(DATABASE, columns = TABLE_COLUMNS)
df.head()